In [44]:
# Langchain Tools
# Arxiv --> For research papers
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
wiki_api_wrapper=WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=200)
wiki_tool=WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
wiki_tool.name

'wikipedia'

In [45]:
arix_api_wrapper=ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=200)
arxiv_tool=ArxivQueryRun(api_wrapper=arix_api_wrapper)
arxiv_tool.name

'arxiv'

In [46]:
tools=[wiki_tool,arxiv_tool]

In [47]:
# Custom Tools (RAG Tools)
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings

In [48]:
loader=WebBaseLoader(web_path="https://docs.smith.langchain.com/")
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content="\n\n\n\n\nGet started with LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith\nLangSmith is a platform for building production-grade LLM applications.\nIt allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.\nObservabilityAnalyze tr

In [50]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
final_doc=text_splitter.split_documents(docs)
final_doc

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet star

In [51]:
vec=FAISS.from_documents(final_doc,embedding=OllamaEmbeddings(model="gemma:2b"))
retriever=vec.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D99A6EEBF0>, search_kwargs={})

In [52]:
from langchain.tools.retriever import create_retriever_tool
my_tool=create_retriever_tool(retriever,"Langsmith-Search","Search any information about Langsmith")
my_tool.name

'Langsmith-Search'

In [53]:
tools=[wiki_tool,arxiv_tool,my_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\DELL\\Generative_AI\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=2, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=2, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='Langsmith-Search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001D9C666B520>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D99A6EEBF0>, s

In [54]:
# Running the tools with Agents and Modules
import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.environ["GROQAI_API_KEY"]=os.getenv("GROQAI_API_KEY")
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=api_key, model_name="mixtral-8x7b-32768")

In [55]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant with access to research tools."),
    ("human", "{input}"),
    ("ai", "{agent_scratchpad}")  # Required for tool interactions
])


In [56]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an AI assistant with access to research tools.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad'], input_types={}, partial_variables={}, template='{agent_scratchpad}'), additional_kwargs={})])
| RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D99A6EFD60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D99A6EF820>, model_kwar

In [57]:
from langchain.agents import AgentExecutor
executer=AgentExecutor(agent=agent,tools=tools,verbose=True)
executer

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an AI assistant with access to research tools.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad'], input_types={}, partial_variables={}, template='{agent_scratchpad}'), additional_kwargs={})])
| RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001D99A6EFD60>, async_client=<groq.resources.chat.co

In [58]:
executer.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...
(

> Finished chain.


{'input': 'Tell me about Langsmith', 'output': '('}